In [ ]:
%pip install gurobipy

  Using cached gurobipy-12.0.0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (14.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

In [24]:
# Data input
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
demand = [47000, 49000, 52000, 35000, 31000, 22000, 26000, 34000, 39000, 41000, 45000, 46000]
production_capacity = 25 * 9  # Volume of working hours per person per month (excluding overtime)
overtime_limit = 16  # Overtime limit pro month
unit_production_time = 2.5  # Work hours per unit of product
initial_inventory = 2800  # Initial inventory
inventory_cost = 10  # Monthly cost per unit of inventory
hire_cost = 850  # Hiring cost pro person
layoff_cost = 1400  # Layoff cost pro person
overtime_cost = 18  # Overtime hourly wage
regular_cost = 16  # regular hourly wage
material_cost = 25  # Material cost per unit
third_party_cost = 72  # Third party cost per unit
selling_price = 80  # Selling price per unit
storage_limit = 25000  # Storage limit
delay_cost = 13  # Delay cost per unit
max_hires = 100  # Maximum number of hires per month

# Model initialization
model = gp.Model("Production Planning")

# Decision variable
production = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="production")
overtime = model.addVars(months, lb=0, vtype=GRB.CONTINUOUS, name="overtime")
inventory = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="inventory")
third_party = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="third_party")
hiring = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="hiring")
layoff = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="layoff")
delay = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="delay")
employees = model.addVars(months, lb=0, vtype=GRB.INTEGER, name="employees")

# Initial condition
model.addConstr(employees["Jan"] == 200)

# Constraint
for i,month in enumerate(months):
    production_hours = production[month] * unit_production_time
    model.addConstr(production_hours == employees[month] * production_capacity + overtime[month])  # Productivity constraints
    model.addConstr(overtime[month] <= employees[month] * overtime_limit)  # Over time constraints
    model.addConstr(inventory[month] <= storage_limit)  # Inventory constraints
    model.addConstr(hiring[month] <= max_hires)  # Hiring constraints 

    # Employee change constraints
    if i > 0:
        prev_month = months[i - 1]
        model.addConstr(
            employees[month] == employees[prev_month] + hiring[prev_month] - layoff[prev_month]
        )

    # Inventory balance constraints
    if i > 0:
        prev_month = months[i - 1]
        model.addConstr(
            inventory[month] == production[month] + third_party[month] + inventory[prev_month]
            - delay[prev_month] - demand[i]
        )
        
    # Delay balance constraints
    if i > 0:
        prev_month = months[i - 1]
        model.addConstr(
            delay[month] == delay[prev_month] + demand[i] - production[month] - third_party[month] - inventory[prev_month]
        )
        
    # Inventory balance constraints(For Jan)
    if i == 0:
        model.addConstr(
            inventory[month] == production[month] + initial_inventory + third_party[month] - demand[i] - delay[month]
        )
    
# Production constraints
model.addConstr(sum((production[m] + third_party[m]) for m in months) == sum(demand[m] for m in range(len(months))))

# Target funciton
revenue = sum(selling_price * (production[m] + third_party[m]) for m in months)
costs = sum(
    regular_cost * employees[m] * production_capacity +
    overtime_cost * overtime[m] +
    hiring[m] * hire_cost +
    layoff[m] * layoff_cost +
    inventory[m] * inventory_cost +
    third_party[m] * third_party_cost +
    delay[m] * delay_cost +
    material_cost * production[m]
    for m in months
)
profit = revenue - costs
model.setObjective(profit, GRB.MAXIMIZE)

# Optimization
model.optimize()

# Output
if model.status == GRB.OPTIMAL:
    print("Optimal Annual Profit:", model.objVal)
    for month in months:
        print(f"{month} - Production: {production[month].x}, Overtime: {overtime[month].x}, "
              f"Inventory: {inventory[month].x}, Third Party: {third_party[month].x}, "
              f"Hiring: {hiring[month].x}, Layoff: {layoff[month].x}, Employees: {employees[month].x}, "
              f"Delay: {delay[month].x}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.3 LTS")

CPU model: Intel(R) Xeon(R) Gold 5218 CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads

Optimize a model with 84 rows, 96 columns and 267 nonzeros
Model fingerprint: 0xa5613715
Variable types: 12 continuous, 84 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [8e+00, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+05]
Presolve removed 55 rows and 65 columns
Presolve time: 0.00s
Presolved: 29 rows, 31 columns, 69 nonzeros
Variable types: 0 continuous, 31 integer (0 binary)
Found heuristic solution: objective 5307930.0000
Found heuristic solution: objective 5309536.0000

Root relaxation: objective 6.001370e+06, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth